In [2]:
# highlight_id, {'field_name: value'}

rdd_1 = sc.parallelize([(1, {'hs/college': 0, 'music': 0}),
                       (2, {'hs/college': 1, 'music': 1}),
                       (3, {'hs/college': 0, 'music': 1}),
                       (4, {'hs/college': 1, 'music': 0})])

rdd_2 = sc.parallelize([(1, {'premium': 0, 'football': 1}),
                       (2, {'premium': 1, 'football': 1}),
                       (3, {'premium': 1, 'football': 1}),
                       (4, {'premium': 1, 'football': 0})])


In [3]:
rdd_full = rdd_1.join(rdd_2)

In [5]:
def add_dicts(a, b):
    c = a.copy()
    c.update(b)
    return c

In [6]:
rdd_full = rdd_full.map(lambda x: (x[0], add_dicts(x[1][0],x[1][1])))

In [7]:
rdd_full.take(1)

[(1, {'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0})]

In [8]:
ids = rdd_full.map(lambda x: x[0]).distinct().collect()

In [9]:
id_matrix = sc.parallelize(ids)

In [10]:
id_matrix.take(1)

[1]

In [11]:
id_matrix = id_matrix.flatMap(lambda x: [(x, i) for i in ids])

In [13]:
id_matrix.take(10)

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2)]

In [17]:
id_joined = id_matrix\
    .join(rdd_full)\
    .map(lambda x: (x[1][0], (x[0], x[1][1])))\
    .join(rdd_full)\
    .map(lambda x: ((x[0], x[1][0][0]), (x[1][1], x[1][0][1])))

In [19]:
id_joined.take(10)

[((1, 1),
  ({'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0},
   {'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0})),
 ((1, 2),
  ({'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0},
   {'football': 1, 'hs/college': 1, 'music': 1, 'premium': 1})),
 ((1, 3),
  ({'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0},
   {'football': 1, 'hs/college': 0, 'music': 1, 'premium': 1})),
 ((1, 4),
  ({'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0},
   {'football': 0, 'hs/college': 1, 'music': 0, 'premium': 1})),
 ((2, 1),
  ({'football': 1, 'hs/college': 1, 'music': 1, 'premium': 1},
   {'football': 1, 'hs/college': 0, 'music': 0, 'premium': 0})),
 ((2, 2),
  ({'football': 1, 'hs/college': 1, 'music': 1, 'premium': 1},
   {'football': 1, 'hs/college': 1, 'music': 1, 'premium': 1})),
 ((2, 3),
  ({'football': 1, 'hs/college': 1, 'music': 1, 'premium': 1},
   {'football': 1, 'hs/college': 0, 'music': 1, 'premium': 1})),
 ((2, 4),
  ({'football': 1, 'hs/c

In [26]:
def dist(a, b):
    dist = 0.
    for key in a:
        dist += abs(a[key]-b[key])
    return dist

In [27]:
distances = id_joined\
    .map(lambda x: (x[0][0], [(x[0][1],dist(x[1][0], x[1][1]))]))\
    .reduceByKey(lambda a,b: a+b)

In [28]:
distances.map(lambda x: (

[(1, [(1, 0.0), (2, 3.0), (3, 2.0), (4, 3.0)]),
 (2, [(1, 3.0), (2, 0.0), (3, 1.0), (4, 2.0)]),
 (3, [(1, 2.0), (2, 1.0), (3, 0.0), (4, 3.0)]),
 (4, [(1, 3.0), (2, 2.0), (3, 3.0), (4, 0.0)])]